In [ ]:
# install necessary packages, if not readily available in underlying python installation 
!pip install xlsxwriter

In [ ]:
#import necessary packages 
import pyodbc 
import pandas as pd
import getpass
from datetime import datetime, timezone
from enum import IntEnum
from datetime import date, datetime, timedelta, timezone
import sys
import math
import xlsxwriter
import numpy as np
import mssqlprofilerpython
#import mypy
#import mssqlprofilerpython2




In [ ]:
#Connect to the data base: Change values as per requirment 
connection_string = 'Server=tcp:bbyc-sqlsv-prod-database-stg-005.database.windows.net,1433;Initial Catalog=bbyc-sqldb-imidiapf-stg-001;Persist Security Info=False;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;'

# Replace 'your_username' and 'your_password' with your actual AD username and password
ad_username = 'arbanerj@bbyfs.ca' #change it to your username 
ad_password = getpass.getpass() #this will ask for your password

modified_connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={connection_string.split(";")[0].split("=")[1]};DATABASE={connection_string.split(";")[1].split("=")[1]};Authentication=ActiveDirectoryPassword;UID={ad_username};PWD={ad_password};'

# Establish the connection
connection = pyodbc.connect(modified_connection_string)
print('connected ...')

In [ ]:
# this is a helper function which is to get this system's date time 
def getDateTimeAsString():

  now = datetime.now(timezone.utc)
  # Convert to local timezone for display
  local_time = now.astimezone()
  date_time_string = local_time.strftime("%Y-%m-%d %H:%M:%S%z")
  return date_time_string

# Print the current date and time with timezone
print(getDateTimeAsString())

In [ ]:
#DataSet Summary Report generation 
# Below are the parameters to deliver to the class.Chnage it as per requirments 

da_params = {
            'SYSTEM_NAME': 'Account Addresses', # CHANGE THIS ONE AS PER REQUIRMENT
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'address',  # CHANGE THIS ONE AS PER REQUIRMENT
            'PROVIDED_BY': 'N/A',  # CHANGE THIS ONE AS PER REQUIRMENT
            'CONTENT': 'Account Addresses Data',  # CHANGE THIS ONE AS PER REQUIRMENT
            'NOTES': 'utf-8',  # CHANGE THIS ONE AS PER REQUIRMENT
            'META_DATA_PRESENT': 'Y',  # CHANGE THIS ONE AS PER REQUIRMENT
            'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address',  # This is the path where your data set summary will get saved, consider this one as 
            # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.  
            'WHO': 'Arpan Banerjee', # CHANGE THIS ONE AS PER REQUIRMENT
            'WHEN': getDateTimeAsString(), # it is remote desktop  date time 
            'WHERE': 'Kolkata-IN', # CHANGE THIS ONE AS PER REQUIRMENT
            'HOW': 'Python pyodbc connector',
            'FILTER': "No filter has been applied"
}



In [ ]:
# run for data set summary report 
datasetSummary =mssqlprofilerpython.DatasetSummary(connection, da_params)
datasetSummary.getSummary()

In [ ]:
# Attribute Summary and Attribute detail report generation  Report
# Below are the parameters to deliver to the class.Chnage it as per requirments 
attr_params = {
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'address',  # CHANGE THIS ONE AS PER REQUIRMENT
            'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address',  # This is the path where your data set summary will get saved, consider this one as 
            # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.
            'WHEN':  getDateTimeAsString(),
            'FILTER': "no filter has been applied"

}


In [ ]:
# run for attribute summary and detail report 
attributesummary= mssqlprofilerpython.AttributeSummary(connection, attr_params)
attributesummary.saveResultToCSV()

In [ ]:
# create separate list of columns for date and non date attribute as well as a list of all atrribute. 
# manualy adjust the location of previously created file attribute_summary 
import csv

def read_date_column(filename, filter_column, filter_values, target_column):

  date_values = []
  non_date_values =[]
  with open(filename, 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      if row[filter_column] in filter_values:
        date_values.append(row[target_column])
      else: 
        non_date_values.append(row[target_column])
     
  
  return date_values,non_date_values 

# donot forget to change the file name with the file path where we are palcing all our csv file with the attribute summary file name. 
filename = "C:\\Users\\arbanerj_a\\membership_address\\bbyc-sqldb-imidiapf-stg-001_membership_address_ATTRIBUTE_SUMMARY.csv"
filter_column = "DATA TYPE"
filter_values = ["date", "datetime"]
target_column = "COLUMN NAME"

date_columns, non_date_columns = read_date_column(filename, filter_column, filter_values, target_column)
complete_column_lst = date_columns + non_date_columns
print(date_columns)
print(non_date_columns)
print(complete_column_lst)



In [ ]:
### HISTOGRAM GENERATION FOR DATE COLUMN
### update values carefully 
### This will genearte monthly quarterly, yearly for specified time frame 
#bring

monthly_params ={
    'PERIOD_TYPE': mssqlprofilerpython.PeriodType.MONTHLY, # keep it as it is 
    'NUMBER_OF_THE_PERIOD': None,# keep it as it is 
    'START_DATE': '2004-09-29', # CHANGE THIS ONE AS PER REQUIRMENT,
    #CHECK THE MINIMUM DATE FROM date and datetime attributes from the saved ATTRIBUTE DETAIL AND TAKE DECISSION ON IT 
    'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
    'SCHEMA_NAME': 'membership', # CHANGE THIS ONE AS PER REQUIRMENT
    'TABLE_NAME': 'address', # CHANGE THIS ONE AS PER REQUIRMENT
    'COLUMN_NAME': 'MEMBERSHIP_START_DATE', # CHANGE THIS ONE AS PER REQUIRMENT
    'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # CHANGE THIS ONE AS PER REQUIRMENT
    'WHEN': getDateTimeAsString(), # CHANGE THIS ONE AS PER REQUIRMENT
    'FILTER': "No filter has been applied"  # CHANGE THIS ONE AS PER REQUIRMENT
}
quarterly_params ={
    'PERIOD_TYPE': mssqlprofilerpython.PeriodType.QUARTERLY, # keep it as it is 
    'NUMBER_OF_THE_PERIOD': None, # keep it as it is 
    'START_DATE': '2004-09-29', # CHANGE THIS ONE AS PER REQUIRMENT,
     #CHECK THE MINIMUM DATE FROM date and datetime attributes from the saved ATTRIBUTE DETAIL AND TAKE DECISSION ON IT 
    'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
    'SCHEMA_NAME': 'membership', # CHANGE THIS ONE AS PER REQUIRMENT
    'TABLE_NAME': 'address', # CHANGE THIS ONE AS PER REQUIRMENT
    'COLUMN_NAME': 'MEMBERSHIP_START_DATE', # CHANGE THIS ONE AS PER REQUIRMENT
    'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # This is the path where your data set summary will get saved, consider this one as 
    # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.
    'WHEN': getDateTimeAsString(), # CHANGE THIS ONE AS PER REQUIRMENT
    'FILTER': "No filter has been applied"  # CHANGE THIS ONE AS PER REQUIRMENT
}
yearly_params ={
    'PERIOD_TYPE': mssqlprofilerpython.PeriodType.YEARLY, # keep it as it is 
    'NUMBER_OF_THE_PERIOD': None, # keep it as it is  # CHANGE THIS ONE AS PER REQUIRMENT,
    'START_DATE': '2004-09-29', # CHANGE THIS ONE AS PER REQUIRMENT,
     #CHECK THE MINIMUM DATE FROM date and datetime attributes from the saved ATTRIBUTE DETAIL AND TAKE DECISSION ON IT     'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
    'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
    'SCHEMA_NAME': 'membership', # CHANGE THIS ONE AS PER REQUIRMENT
    'TABLE_NAME': 'address', # CHANGE THIS ONE AS PER REQUIRMENT
    'COLUMN_NAME': 'MEMBERSHIP_START_DATE', # CHANGE THIS ONE AS PER REQUIRMENT
    'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # This is the path where your data set summary will get saved, consider this one as 
    # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.    'WHEN': getDateTimeAsString(), # CHANGE THIS ONE AS PER REQUIRMENT
    'WHEN': getDateTimeAsString(), # CHANGE THIS ONE AS PER REQUIRMENT
    'FILTER': "No filter has been applied"  # CHANGE THIS ONE AS PER REQUIRMENT
}


In [ ]:
# run histogram 

for item in date_columns:
  new_column = {'COLUMN_NAME': item}
  monthly_params.update(new_column)
  quarterly_params.update(new_column)
  yearly_params.update(new_column)
  month_wise_distribution =  mssqlprofilerpython.HistogramGeneration(connection,monthly_params)
  month_wise_distribution.saveResultToCSV()

  quarter_wise_distribution =  mssqlprofilerpython.HistogramGeneration(connection,quarterly_params)
  quarter_wise_distribution.saveResultToCSV()

  year_wise_distribution =  mssqlprofilerpython.HistogramGeneration(connection,yearly_params)
  year_wise_distribution.saveResultToCSV()
    


In [ ]:
#Histogram enhancer 
#chnage parameter carefully 
enhancer_params = {

    'COLUMNS': date_columns,
    'START_YEAR': 2004, # CHANGE THIS ONE AS PER REQUIRMENT, SAME CHANGES REQUIRED WHERE THE MINIMUM DATE 
    # change this Start_year  as the minimum date you recived from attribute detail csv. 
    'START_MONTH': 1 , # CHANGE THIS ONE AS PER REQUIRMENT
    'YEARS_TO_COVER': 21, # CHANGE THIS ONE AS PER REQUIRMENT
    #'PROFILE_DATE': '2023-03-10', # CHANGE THIS ONE AS PER REQUIRMENT
    'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # This is the path where your data set summary will get saved, consider this one as 
    # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.    
    'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
    'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
    'TABLE_NAME': 'address' # CHANGE THIS ONE AS PER REQUIRMENT

}


In [ ]:
# call the histogram enhancer 
enahncer =mssqlprofilerpython.HistogramEnhancer(enhancer_params)
enahncer.caller()

In [ ]:
# Generate distribution report for date attributes 
# change below parameters carefully 
date_distributionparams = {
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'address',  # CHANGE THIS ONE AS PER REQUIRMENT
            'COLUMN_LIST': date_columns, #coming from previous helper function
            'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # This is the path where your data set summary will get saved, consider this one as 
             # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.    
            'WHEN': getDateTimeAsString(),
            'FILTER': f"No filter has been applied"  # CHANGE THIS ONE AS PER REQUIRMENT

}

In [ ]:
# run the distribution report for date attributes 
groupbycount=mssqlprofilerpython.DateDimensionalFrquencey(connection,date_distributionparams)
groupbycount.loopAndCount()

In [ ]:
# Generate distribution report for non date attributes 
# change below parameters carefully 
non_date_distributionparams = {
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'address',  # CHANGE THIS ONE AS PER REQUIRMENT
            'COLUMN_LIST': non_date_columns, #coming from previous helper function
            'FILE_PATH': 'C:\\Users\\arbanerj_a\\membership_address', # This is the path where your data set summary will get saved, consider this one as 
             # a path, where intermediate csv will get saved. For best practice create a folder in your current directory with the table name.    
            'WHEN': getDateTimeAsString(),
            'FILTER': f"No filter has been applied"  # CHANGE THIS ONE AS PER REQUIRMENT

}

In [ ]:
# run the distribution report for date attributes 
groupbycount=mssqlprofilerpython.DimensionalFrquencey(connection,non_date_distributionparams)
groupbycount.loopAndCount()

In [1]:
# Generate the excel report by combining csv files 
#chnage the parameters carefully 
complete_column_lst =['created_date', 'updated_date', 'address_id', 'address_line_1', 'address_line_2', 'city', 'country_code', 'country_name', 'created_by', 'phone_number', 'postal_code', 'region_code', 'region_name', 'suite', 'updated_by', 'address_type_id', 'membership_id']
csv2excel_params = {
            'SOURCE_FOLDER': 'C:\\Users\\arbanerj_a\\membership_address', # CHANGE THIS ONE AS PER REQUIRMENT
            # Source folder is the folder where we have saved all intermediate csv reports  
            'DESTINATION_FOLDER': 'C:\\Users\\arbanerj_a\\membership_address\\Report', # CHANGE THIS ONE AS PER REQUIRMENT
            #Destination is the  folder  where we will save the excel output 
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'membership',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'address',  # CHANGE THIS ONE AS PER REQUIRMENT
            'COLUMNS': complete_column_lst, # CHANGE THIS ONE AS PER REQUIRMENT
            'DATE': '2024-03-18',  # CHANGE THIS ONE AS PER REQUIRMENT
             # put it as todayas date yyyy-mm-dd
            'START_ROW_FOR_DISTINCT': 22 , # CHANGE THIS ONE AS PER REQUIRMENT
            'TOP_DISTINCT': 21 ,  # CHANGE THIS ONE AS PER REQUIRMENT
            'YEARS_TO_BE_COVERED': 10 # change this one to change your histigram behaviour 
}

In [2]:
import mysqlpy

ce = mysqlpy.CsvToExcel(csv2excel_params)
ce.startConverting()

column name:  address_id
replacing value...........................................
after grp by
    VALUE  COUNT       %
0   14104      1  0.0002
1    1819      1  0.0002
18   6893      1  0.0002
17   6777      1  0.0002
16  66360      1  0.0002
15  65577      1  0.0002
14  65164      1  0.0002
13  60154      1  0.0002
12  59761      1  0.0002
11  56345      1  0.0002
10  52889      1  0.0002
9   51540      1  0.0002
8   46498      1  0.0002
7   40091      1  0.0002
6   33003      1  0.0002
5   27653      1  0.0002
4   26025      1  0.0002
3   22704      1  0.0002
2   19076      1  0.0002
19     80      1  0.0002
2nd time:     VALUE  COUNT       %
0   14104      1  0.0002
1    1819      1  0.0002
18   6893      1  0.0002
17   6777      1  0.0002
16  66360      1  0.0002
15  65577      1  0.0002
14  65164      1  0.0002
13  60154      1  0.0002
12  59761      1  0.0002
11  56345      1  0.0002
10  52889      1  0.0002
9   51540      1  0.0002
8   46498      1  0.0002
7   40091      1  

In [ ]:
#close datbase connection 
connection.close()